# Clone repositories

In [10]:
import csv
import git

projects_name = []
with open('repos.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        #git.Repo.clone_from(row['url'], row['name'])
        print(row['name'] + " repository has been cloned")
        projects_name.append(row['name'])
    print("cloned projects names are: ", projects_name)

Anki-Android repository has been cloned
tutanota repository has been cloned
OpenHub repository has been cloned
snapcast repository has been cloned
Conversations repository has been cloned
AntennaPod repository has been cloned
AmazeFileManager repository has been cloned
NetGuard repository has been cloned
qksms repository has been cloned
AdAway repository has been cloned
stk-code repository has been cloned
status-react repository has been cloned
andOTP repository has been cloned
android repository has been cloned
cloned projects names are:  ['Anki-Android', 'tutanota', 'OpenHub', 'snapcast', 'Conversations', 'AntennaPod', 'AmazeFileManager', 'NetGuard', 'qksms', 'AdAway', 'stk-code', 'status-react', 'andOTP', 'android']


# Extract traditional smells
returns a dictionary of list, with key represents a traditional smell and value represents list of classes that contain the smell

In [11]:
import os
import subprocess
import configparser
import pandas as pd
import re
import shutil
import time
import sys

def extract_traditional_smells(project_name):
    print("extracting traditional smells from repository: " + project_name)
    %cd ./resources
    subprocess.call(['java', '-jar', './DECOR_JAVA.jar', "../" + project_name, project_name, ""])
    %cd ../

    # check if the result of detection is generated
    if os.path.isdir('./TEMP'):

        # dictionary of list, key represents a traditional smell and value represents list of classes that contain the smell
        smell_classes_dict = dict()

        # iterate through each generated file
        result_files_path = "./TEMP/" + project_name
        for filename in os.listdir(result_files_path):

            # read the content of the file
            file_path = result_files_path + "/" + filename
            with open(file_path) as file:
                file_content = '[dummy_section]\n' + file.read()
            config = configparser.ConfigParser()
            config.read_string(file_content)

            # find all smelly test classes that contain traditional smell
            smelly_classes = []
            for key, value in config.items('dummy_section'):
                if re.match("^[0-9]+\.[0-9]+\.[a-zA-Z]+-\d$", key):
                    class_name = value.split(".")[-1]
                    if "test" in class_name.lower() or "tests" in class_name.lower():
                        smelly_classes.append(class_name)
            # map smell and classes containing the smell
            if smelly_classes:
                smell_name = filename.split(" ")[-1].split(".")[0]
                smell_classes_dict[smell_name] = smelly_classes

        # delete result of detection
        try:
            shutil.rmtree("./TEMP/")
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        finally:
            print("Traditional smells extracted for project: ", project_name)
            return smell_classes_dict

    else:    
        print("ERROR: extracting traditionial smells failed")
        return None

# Extract test smells

In [12]:
import subprocess
import glob
import pandas as pd
import os

def delete_testFileDetector_output_files():
    files_to_delete = glob.glob('Output_*_*.csv')
    for file in files_to_delete:
        os.remove(file)
    if os.path.exists("Log.txt"):
        os.remove("Log.txt")
        
def delete_testFileMapping_output_files(file_name):
    os.remove(file_name)
    os.remove("test_file_path.txt")

def extract_test_smells(project_name, output_file_name):
    print("extracting test smells from repository: " + project_name)
    
    # TestFileDetector
    subprocess.call(['java', '-jar', './resources/TestFileDetector.jar', project_name])
    file_name = glob.glob('Output_Class_*.csv')[0]
    df = pd.read_csv(file_name)
    a = list(df['FilePath'])
    # if no test file has been detected in the project
    if not a:
        print("No test file detected in project: ", project_name, "\n")  
        delete_testFileDetector_output_files()
        return
    b = '\n'.join(str(e) for e in a)
    with open('test_file_path.txt', 'w') as f:
        f.write(b)
    # delete output files
    delete_testFileDetector_output_files()
    
    # TestFileMapping
    subprocess.call(['java', '-jar', './resources/TestFileMapping.jar', "test_file_path.txt"])   
    file_name = glob.glob('Output_TestFileMappingDetection_*.csv')[0]
    df = pd.read_csv(file_name)
    df = df[['App','TestFilePath', 'ProductionFilePath']]
    df = df.dropna()
    if df.empty:
        print("No production files found for given test files", "\n")
        # delete output files
        delete_testFileMapping_output_files(file_name)
        return
    df.to_csv("test_production_file_path.csv", index=False, header=False) 
    # delete output files
    delete_testFileMapping_output_files(file_name)
    
    # TestSmellDetection
    subprocess.call(['java', '-jar', './resources/TestSmellDetector.jar', "test_production_file_path.csv"])
    file_name = glob.glob('Output_TestSmellDetection_*.csv')[0]
    df = pd.read_csv(file_name)
    df = df.drop(['App', 'TestFilePath', 'ProductionFilePath', 'RelativeTestFilePath', 'RelativeProductionFilePath'], axis = 1)
    df.to_csv(output_file_name, index = False)
    print("Test smells extracted for project: ", project_name)
    # delete output files
    os.remove("test_production_file_path.csv")
    os.remove(file_name)

# Extract and combine test and traditional smells

In [13]:
TEST_SMELLS = ['Assertion Roulette', 'Conditional Test Logic', 'Constructor Initialization', 'Default Test', 'EmptyTest', 'Exception Catching Throwing', 'General Fixture', 'Mystery Guest', 'Print Statement', 'Redundant Assertion', 'Sensitive Equality', 'Verbose Test', 'Sleepy Test', 'Eager Test', 'Lazy Test', 'Duplicate Assert', 'Unknown Test', 'IgnoredTest', 'Resource Optimism', 'Magic Number Test', 'Dependent Test']

TRADITIONAL_SMELLS = ['ComplexClass', 'FunctionalDecomposition', 'BaseClassKnowsDerivedClass', 'MessageChains', 'SpaghettiCode', 'LongMethod', 'LazyClass', 'LongParameterList', 'SwissArmyKnife', 'RefusedParentBequest', 'AntiSingleton', 'Blob', 'TraditionBreaker', 'BaseClassShouldBeAbstract', 'ClassDataShouldBePrivate', 'ManyFieldAttributesButNotComplex', 'SpeculativeGenerality', 'LargeClass']


In [14]:
import os

def extract_smells(project_name, combined_smells_output_file):
    # extract test and traditional smells
    test_smells_output_file = "test_smells.csv"
    extract_test_smells(project_name, test_smells_output_file)
    if not os.path.isfile(test_smells_output_file):
        return
    smell_classes_dict = extract_traditional_smells(project_name)
    
    # combine test and traditional smells into one single dataframe
    df = pd.read_csv(test_smells_output_file)
    for traditional_smell in TRADITIONAL_SMELLS:
        df[traditional_smell] = False
    if smell_classes_dict:
        for smell, classes in smell_classes_dict.items():
            for smelly_class in classes:
                smelly_class_name = smelly_class + ".java"
                indexes = df.index[(df['TestClass'] == smelly_class_name)].tolist()
                if indexes:
                    for index in indexes:
                        df.at[index, smell] = True
                # if test class does not exist among smelly test files, add a new row in the dataframe
                else:
                    temp = [False] * (len(TEST_SMELLS) + len(TRADITIONAL_SMELLS))
                    new_row = [smelly_class_name] + temp
                    df.loc[len(df.index)] = new_row
                    df.at[len(df.index) - 1, smell] = True       
    df = df.drop(['TestClass'], axis = 1)
    df = df.astype(int)
    if os.path.isfile(combined_smells_output_file):
        df1 = pd.read_csv(combined_smells_output_file)
        df = pd.concat([df1, df], ignore_index=True, sort=False)
    df.to_csv(combined_smells_output_file, index = False)
    print("finished generating test and traditional smells for project: ", project_name, "\n")
    
    # delete output files
    os.remove(test_smells_output_file) 

# Compute co-occurrence matrix

In [15]:
import numpy as np

def compute_cooccurrence_matrix(csv):
    df = pd.read_csv(csv)
    full_cooccurrence_matrix = df.T.dot(df)
    # compute co-occurrence matrix in percentage
    # https://stackoverflow.com/questions/20574257/constructing-a-co-occurrence-matrix-in-python-pandas
    full_cooccurrence_matrix_diagonal = np.diagonal(full_cooccurrence_matrix)
    with np.errstate(divide='ignore', invalid='ignore'):
        full_cooccurrence_matrix_percentage = np.nan_to_num(np.true_divide(full_cooccurrence_matrix, full_cooccurrence_matrix_diagonal[:, None]))
    # https://stackoverflow.com/questions/16444930/copy-upper-triangle-to-lower-triangle-in-a-python-matrix
    full_cooccurrence_matrix_percentage = np.triu(full_cooccurrence_matrix_percentage)
    full_cooccurrence_matrix_percentage = full_cooccurrence_matrix_percentage + full_cooccurrence_matrix_percentage.T - np.diag(np.diag(full_cooccurrence_matrix_percentage))
    
    full_cooccurrence_matrix_percentage = pd.DataFrame(data = full_cooccurrence_matrix_percentage, columns = TEST_SMELLS + TRADITIONAL_SMELLS, index = TEST_SMELLS + TRADITIONAL_SMELLS)
    # transform full co-occurrence matrix into co-occurrence matrix between test and traditional smells
    co_occurrence_matrix_percentage = full_cooccurrence_matrix_percentage.drop(TRADITIONAL_SMELLS, axis=1).drop(TEST_SMELLS)
    co_occurrence_matrix_percentage.to_csv("co-occurrence matrix percentage.csv")
    
    print("co-occurrence matrix generated for all projects \n")

# Test

## Compute co-occurrence matrix for latest version of each project

In [440]:
combined_smells_output_file = "combined_smells.csv"
for project_name in projects_name:
    extract_smells(project_name, combined_smells_output_file)
compute_cooccurrence_matrix(combined_smells_output_file)

hello
extracting test smells from repository: muzei
No test file detected in project:  muzei 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: tutanota
Test smells extracted for project:  tutanota
extracting traditional smells from repository: tutanota
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  tutanota
finished generating test and traditional smells for project:  tutanota 

extracting test smells from repository: OpenHub
No production files found for given test files 

extracting test smells from repository: snapcast
No test file detected in project:  snapcast 

extracting test smells from repository: Conversations
No test f

## Compute co-occurrence matrix for full history version of each project

In [16]:
import csv
import git

combined_smells_output_file = "combined_smells.csv"
with open('repos.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        project_name = row['name']
        repo = git.Repo.clone_from(row['url'], project_name)
        print(project_name + " repository has been cloned")
        for commit in repo.iter_commits():
            repo.git.checkout(commit)
            extract_smells(project_name, combined_smells_output_file)
compute_cooccurrence_matrix(combined_smells_output_file)

Anki-Android repository has been cloned
extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and tra

/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resou

Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-And

finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-And

/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resou

...

finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-And

/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resou

Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-And

finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-Android
finished generating test and traditional smells for project:  Anki-Android 

extracting test smells from repository: Anki-Android
Test smells extracted for project:  Anki-Android
extracting traditional smells from repository: Anki-Android
/Users/vincedu/resources
/Users/vincedu
Traditional smells extracted for project:  Anki-And

GitCommandError: Cmd('git') failed due to: exit code(1)
  cmdline: git checkout 201079b85ed45f482e8c570721a280c695e1042e
  stderr: 'error: Your local changes to the following files would be overwritten by checkout:
	AnkiDroid/src/test/java/com/ichi2/anki/services/NoteServiceTest.java
Please commit your changes or stash them before you switch branches.
Aborting'